In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 2.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73052 sha256=b100823d10415ed2ab44cfe0782605a5c3afc176df2039e3781e91c52ac58ff2
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                         title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
akshaydattatraykhare/diabetes-dataset                       Diabetes Dataset                                      9KB  2022-10-06 08:55:25          18484        524  1.0              
akshaydattatraykhare/data-for-admission-in-the-university   Data for Admission in the University                  4KB  2022-10-27 11:05:45           5375        125  1.0              
thedevastator/jobs-dataset-from-glassdoor                   Salary Prediction                                     3MB  2022-11-16 13:52:31           1640         47  1.0              
meirnizri/covid19-dataset                     

In [ ]:
!kaggle datasets download -d mostafaabla/garbage-classification -p /content/sample_data/ --unzip

 98% 234M/239M [00:01<00:00, 138MB/s]
100% 239M/239M [00:01<00:00, 163MB/s]


In [ ]:
from google.colab import files
files.upload()

Saving preprocessed_data.csv to preprocessed_data.csv


{'preprocessed_data.csv': b'photo,class\nplastic200.jpg,3.0\ncardboard456.jpg,1.0\nmetal104.jpg,2.0\nwhite-glass53.jpg,4.0\ncardboard820.jpg,1.0\nplastic822.jpg,3.0\ngreen-glass46.jpg,4.0\nmetal526.jpg,2.0\nbrown-glass192.jpg,4.0\nbrown-glass318.jpg,4.0\nwhite-glass465.jpg,4.0\npaper650.jpg,1.0\ncardboard525.jpg,1.0\nmetal453.jpg,2.0\ngreen-glass261.jpg,4.0\ncardboard263.jpg,1.0\nmetal710.jpg,2.0\nplastic861.jpg,3.0\npaper214.jpg,1.0\ngreen-glass27.jpg,4.0\npaper822.jpg,1.0\npaper438.jpg,1.0\nplastic288.jpg,3.0\npaper824.jpg,1.0\nplastic582.jpg,3.0\nbrown-glass531.jpg,4.0\nbrown-glass303.jpg,4.0\ncardboard843.jpg,1.0\nmetal507.jpg,2.0\nbrown-glass132.jpg,4.0\ncardboard166.jpg,1.0\nbrown-glass376.jpg,4.0\nplastic462.jpg,3.0\ncardboard560.jpg,1.0\npaper278.jpg,1.0\nmetal98.jpg,2.0\nmetal509.jpg,2.0\nplastic665.jpg,3.0\ncardboard828.jpg,1.0\nmetal123.jpg,2.0\nplastic615.jpg,3.0\nwhite-glass20.jpg,4.0\nwhite-glass268.jpg,4.0\nplastic540.jpg,3.0\nbrown-glass347.jpg,4.0\nplastic815.jpg,3.0\n

In [ ]:
import pandas as pd
df = pd.read_csv("preprocessed_data.csv")
df

,photo,class
0,plastic200.jpg,3.0
1,cardboard456.jpg,1.0
2,metal104.jpg,2.0
3,white-glass53.jpg,4.0
4,cardboard820.jpg,1.0
...,...,...
5581,plastic429.jpg,3.0
5582,paper784.jpg,1.0
5583,brown-glass509.jpg,4.0
5584,cardboard827.jpg,1.0


In [ ]:
import os
import shutil

def moveToAll(neededDirs):
    pathTo   = "./sample_data/toSort/"
    pathFrom = "./sample_data/garbage_classification/"

    if not os.path.isdir(pathTo):
        os.mkdir(pathTo)
    else:
        shutil.rmtree(pathTo)
        os.mkdir(pathTo)

    for folder in neededDirs:
        print(len(os.listdir(pathFrom+folder)))
        for filename in os.listdir(pathFrom+folder):
            shutil.copy2(pathFrom+folder+"/"+filename, pathTo)

neededDirs = ["brown-glass", "cardboard", "green-glass", "plastic", "paper", "white-glass", "metal"]
moveToAll(neededDirs)

607
891
629
865
1050
775
769


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

#output - vector of vectors, labels - vactor
def accuracy(outputs, labels):
    return torch.tensor(torch.sum(outputs.argmax(1) == labels.argmax(1)).item() / len(labels))

#implement Base for Classification
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        #print(batch.shape)
        #print()
        #batch - (img, label)
        self.train()
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        #batch - (img, label)
        self.eval() 
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        

    def validation_epoch_end(self, outputs):
        #outputs - list of dicts of validations 
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print(f"Epoch {epoch+1}: train_loss: {result['train_loss']}, val_loss: {result['val_loss']}, val_acc: {result['val_acc']}")

class ImageClassificationInception(nn.Module):
    def training_step(self, batch):
        #print(batch.shape)
        #print()
        #batch - (img, label)
        self.train() 
        images, labels = batch 
        outputs = self(images) 
        loss1 = F.cross_entropy(outputs, labels)
        loss = loss1
        return loss
    
    def validation_step(self, batch):
        #batch - (img, label)
        self.eval() 
        images, labels = batch 
        outputs = self(images) 
        loss1 = F.cross_entropy(outputs, labels)
        loss = loss1
        acc = accuracy(outputs, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        

    def validation_epoch_end(self, outputs):
        #outputs - list of dicts of validations 
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print(f"Epoch {epoch+1}: train_loss: {result['train_loss']}, val_loss: {result['val_loss']}, val_acc: {result['val_acc']}")

In [ ]:
classDecode = {1:"paper/cardboard", 2: "metal", 3: "plastic", 4: "glass"}
class ResNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet50(pretrained=True)
        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs, len(classDecode.values()))
    
    def forward(self, xb):
        o = self.network(xb)
        return F.softmax(o)

class DenseNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = models.densenet121(pretrained=True)
        self.network.classifier = nn.Linear(1024, len(classDecode.values()))
    
    def forward(self, xb):
        o = self.network(xb)
        return F.softmax(o)

class InceptionNet(ImageClassificationInception):
    def __init__(self):
        super().__init__()
        self.network = models.inception_v3(pretrained=True)
        self.network.aux_logits=False
        num_ftrs = self.network.AuxLogits.fc.in_features
        self.network.AuxLogits.fc = nn.Linear(num_ftrs, len(classDecode.values()))
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs,len(classDecode.values()))
    
    def forward(self, xb):
        o = self.network(xb)
        return F.softmax(o)


class VGGNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.vgg16(pretrained=True)
        # Replace last layer
        self.network.classifier[6] = torch.nn.Linear(4096, len(classDecode.values()))
    
    def forward(self, xb):
        o = self.network(xb)
        return F.softmax(o)

class Net(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=(1,1))
        self.conv2 = nn.Conv2d(16, 16, 3, padding=(1,1))
        self.conv3 = nn.Conv2d(16, 32, 3, padding=(1,1))
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv5 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv6 = nn.Conv2d(64, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32 * 4 * 4 , 128)
        self.fc3 = nn.Linear(128, 6)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = self.pool(F.relu(self.conv6(x)))
        #print(x.shape)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc3(x)
        return F.softmax(x)

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
import cv2
import os
import shutil
import pandas as pd
import os.path
import time

def read_and_resize(filename, grayscale = False, fx= 1, fy=1):
    #read file
    if grayscale:
        img_result = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
        imgbgr = cv2.imread(filename, cv2.IMREAD_COLOR)
        img_result = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2RGB)
    img_result = cv2.resize(img_result, None, fx=fx, fy=fy, interpolation = cv2.INTER_CUBIC)
    return img_result

def getImageFromFolderAndMove(pathToImage, pathNewFolderImages):
    #get image and move produced image to another folder
    out = []
    if not os.path.isdir(pathNewFolderImages):
        os.mkdir(pathNewFolderImages)
    if len(os.listdir(pathToImage)) == 0:
        print("Folder toSort is empty!")
    for filename in os.listdir(pathToImage):
        imgPath = (pathToImage+"/").replace("//", "/")+filename
        out = read_and_resize(imgPath)
        shutil.move(imgPath, (pathNewFolderImages+"/").replace("//", "/")+filename)
        return [out, filename]

def resizeImage(img, size):
    #reshape to the shape of (size, size)
    return cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA).astype("float32")/255

def convertImg(img, device):
    #prepare image to pass it to the model
    return torch.tensor([img.swapaxes(2,1).swapaxes(1,0)]).to(device)

def passToModel(model, image):
    return model(image)

def loadModel(pathTo, device):
    model = to_device(InceptionNet(), device)
    model.load_state_dict(torch.load(pathTo, map_location=device))
    return model

def logInfo(pathToCsv, imgName, accuracy, predict, predictClass):
    if not os.path.isfile(pathToCsv): 
        df = pd.DataFrame({'name': [],
                           'accuracy': [],
                   'class': [],
                   'classname': []})
    else:
        df = pd.read_csv(pathToCsv)
    new_row = pd.DataFrame({'name': [imgName],
                           'accuracy': [accuracy],
                   'class': [predict],
                   'classname': [predictClass]})
    df = df.append(new_row, ignore_index=True)
    df.to_csv(pathToCsv, index=False)


def getResizePass(pathToModel, pathToImage, pathNewFolderImages, csvFilePath, classDecode):
    device = get_default_device()
    model = loadModel(pathToModel, device)
    model.eval()
    folderSize = len(os.listdir(pathToImage))
    for i in range(folderSize):
        print(i/folderSize*100, i, folderSize)
        img, filename = getImageFromFolderAndMove(pathToImage, pathNewFolderImages)
        resImg = convertImg(resizeImage(img, 256), device)

        out = passToModel(model, resImg)
        outMax = list(map(lambda a: "{:.3f}".format(a), out.detach().numpy()[0]))
        outArgmax = out.argmax(1)
        cls = classDecode[int(outArgmax[0]+1)]
        numCls = int(outArgmax[0]+1)
        log = logInfo(csvFilePath, filename, outMax, numCls, cls)
        #print(f"Model predicted class {numCls}, which is a {cls}")
    
def main():
    googlePath = "./drive/MyDrive/"
    googleData = "./sample_data/"
    localPath = "./processing/"
    path = googlePath
    pathToModel = path + "model/inceptionModel.pt"  # pretrained model. It takes RGB images with size 256x256
    imgPath = googleData + "toSort"  # folder with all images to sort
    imgNewPath = googleData + "sorted"  # folder with all sorted images
    csvPath = path + "allPredictionsInception.csv"  # file for the logging
    classDecoder = {1: "paper/cardboard", 2: "metal", 3: "plastic", 4: "glass"}  # all classes

    getResizePass(pathToModel, imgPath, imgNewPath, csvPath, classDecoder)

In [ ]:
main()

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


0.0 0 4494


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.022251891410769914 1 4494
0.04450378282153983 2 4494
0.06675567423230974 3 4494
0.08900756564307966 4 4494
0.11125945705384957 5 4494
0.13351134846461948 6 4494
0.1557632398753894 7 4494
0.1780151312861593 8 4494
0.20026702269692925 9 4494
0.22251891410769914 10 4494
0.24477080551846908 11 4494
0.26702269692923897 12 4494
0.2892745883400089 13 4494
0.3115264797507788 14 4494
0.33377837116154874 15 4494
0.3560302625723186 16 4494
0.37828215398308856 17 4494
0.4005340453938585 18 4494
0.4227859368046284 19 4494
0.4450378282153983 20 4494
0.46728971962616817 21 4494
0.48954161103693816 22 4494
0.511793502447708 23 4494
0.5340453938584779 24 4494
0.5562972852692478 25 4494
0.5785491766800178 26 4494
0.6008010680907877 27 4494
0.6230529595015576 28 4494
0.6453048509123275 29 4494
0.6675567423230975 30 4494
0.6898086337338674 31 4494
0.7120605251446372 32 4494
0.7343124165554071 33 4494
0.7565643079661771 34 4494
0.778816199376947 35 4494
0.801068090787717 36 4494
0.8233199821984869 37 449

In [ ]:
df2 = pd.read_csv("./drive/MyDrive/" +"allPredictionsInception.csv")
df2

,name,accuracy,class,classname
0,white-glass31.jpg,[4.1612767e-04 1.4512229e-03 9.9112445e-01 7.0...,3.0,plastic
1,green-glass363.jpg,[2.4283398e-05 6.6784373e-06 4.0998426e-04 9.9...,4.0,glass
2,brown-glass346.jpg,[4.9570710e-05 5.6657278e-05 5.4305780e-04 9.9...,4.0,glass
3,white-glass69.jpg,[9.6589463e-07 8.3249756e-07 1.8237524e-05 9.9...,4.0,glass
4,brown-glass423.jpg,[5.7626892e-07 2.2739773e-06 8.3575806e-06 9.9...,4.0,glass
...,...,...,...,...
5263,plastic801.jpg,"['0.000', '0.000', '1.000', '0.000']",3.0,plastic
5264,paper523.jpg,"['1.000', '0.000', '0.000', '0.000']",1.0,paper/cardboard
5265,white-glass184.jpg,"['0.000', '0.002', '0.994', '0.004']",3.0,plastic
5266,cardboard290.jpg,"['1.000', '0.000', '0.000', '0.000']",1.0,paper/cardboard
